In [ ]:
%load_ext autoreload
%autoreload 2

# Analytical solution: scan over initial copynumber for different peaks

In this notebook, we consolidate the analytical solution derived in notebook `27_sensitivity_peak3` into functions. We then use these functions to scan over the likely peaks in sensitivity and precision across different starting copynumbers.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import jax.numpy as jnp
from scipy.optimize import root, least_squares
import importlib, warnings
import sympy as sp


from synbio_morpher.utils.modelling.physical import gibbs_K, equilibrium_constant_reparameterisation


### Define scan parameters

In [ ]:
scan_params = {
    'n_species': [2, 3, 4, 5, 6, 7, 10],
    'starting_copynumbers': [1, 2, 5, 10, 50, 100, 500, 1000, 1e4],
    'signal_perturb_mult': [0.1, 0.5, 1, 2, 5, 10, 100]
}

n = scan_params['n_species']
xinit = scan_params['starting_copynumbers']
s = scan_params['signal_perturb_mult']
k_a = 1e6  # 1/(M*s)

K = equilibrium_constant_reparameterisation(0, initial=200)

### Functions for analytical solution

In [ ]:
# helpers
def get_kd(k_a, K):
    return k_a / K

# derivation


# root-finding

# root selection

Derivation of steady states for analytical solution before perturbation

$x + (n+1) K x^2 = c_0$ --------> $x = \frac{-1 ± \sqrt{1 - 4((n+1)K)(-c_0)}}{2((n+1)K)}$

In [ ]:
def calc_x_pre(c_0, K, n):
    top = jnp.sqrt(1 - 4 * (n+1) * K * (-c_0))
    denom = 2 * (n+1) * K

    x_plus = jnp.divide(-1 + top, denom)
    x_minus = jnp.divide(-1 - top, denom)

    return x_plus, x_minus

In [18]:
import math
n = 4
math.factorial(n) / math.factorial(n - 1), np.sum(np.triu(np.ones((n, n))))

(4.0, np.float64(10.0))

In [20]:
n*n - ((n * n - n) / 2)

10.0

Finding steady states as roots of perturbed system 



For i = range(n - 1) | i!=1, j = range(n - 1) | j>=i:

$y_{11} = K a^2$ <------ * 1

$y_{1i} = K ab$  <------ * (n - 1)

$y_{ij} = K b^2$ <------ 





#### Change the conservation equations

This then changes the conservation equations to the following for the signal species $RNA_1$:

$a + 2 y_{11} + (n - 1) y_{1i} = s c_1 + (n + 1) K c_1^2$

and for the non-perturbed species $RNA_2$ and $RNA_3$:

$RNA_2$:  $b + 2 y_{22} + y_{12} + y_{23} = c_1 + 4 K c_1^2$

$RNA_3$:  $b + 2 y_{33} + y_{13} + y_{23} = c_1 + 4 K c_1^2$


Substituting out the paired species, we get coupled pair of algebraic equations:

$a + 2 K a^2 + K a b + K a b = 2 c_1 + 4 K c_1^2$

$b + 2 K b^2 + K a b + K b^2 = c_1 + 4 K c_1^2$

We futher reduce these to:

$a + 2 K a^2 + 2 K a b = 2 c_1 + 4 K c_1^2$

$b + 3 K b^2 + K a b = c_1 + 4 K c_1^2$

#### Solving polynomial coupled equation

$a + 2 K a^2 + 2 K a b - (2 c_1 + 4 K c_1^2) = b + 3 K b^2 + K a b - (c_1 + 4 K c_1^2)$

$a + 2 K a^2 + 2 K a b - c_1 = b + 3 K b^2 + K a b$

$a + 2 K a^2 + K a b - c_1 = b + 3 K b^2$

$2 K a^2 - 3 K b^2 + K a b + a - b - c_1 = 0$

We can use python packages that handle such polynomial system calculations such as `sympy` for finding $a$ and $b$, given that we know $K$ and $c_1$ already.



In [ ]:
def F(vars):
    a, b = vars
    f1 = a + 2*K_2*a*a + 2*K_2*a*b - 2*c1 - 4*K_2*c1**2
    f2 = b + 3*K_2*b*b + K_2*a*b - c1 - 4*K_2*c1**2
    return np.array([f1, f2])

initial_guess = np.array([2*c1, c1], dtype=np.float64)

# 1) scipy.optimize.root
sol_root = root(F, initial_guess, method='hybr')
sol_root_lm = root(F, initial_guess, method='lm')

# 2) least_squares with positivity bounds
sol_ls = least_squares(F, initial_guess, bounds=(0, np.inf))

print("root (hybr):", sol_root.x, "success:", sol_root.success)
print("root (lm):", sol_root_lm.x, "success:", sol_root_lm.success)
print("least_squares:", sol_ls.x, "success:", sol_ls.success)

# 3) Elimination -> polynomial in b (requires sympy)
roots_from_elim = None
if importlib.util.find_spec("sympy") is not None:
    a_sym, b_sym, K_sym, c1_sym = sp.symbols('a b K_2 c1')
    f1_sym = a_sym + 2*K_sym*a_sym**2 + 2*K_sym*a_sym*b_sym - 2*c1_sym - 4*K_sym*c1_sym**2
    f2_sym = b_sym + 3*K_sym*b_sym**2 + K_sym*a_sym*b_sym - c1_sym - 4*K_sym*c1_sym**2
    poly_b = sp.resultant(f1_sym, f2_sym, a_sym)
    poly_b_num = sp.expand(poly_b.subs({K_sym:K_2, c1_sym:c1}))
    coeffs = sp.Poly(poly_b_num, b_sym).all_coeffs()
    coeffs = np.array([np.float64(cc) for cc in coeffs], dtype=np.float64)
    roots_b = np.roots(coeffs)
    a_from_b = []
    for rb in roots_b:
        if abs(rb) > 1e-12:
            # a_val = (c1 - rb - 3*K_2*rb**2) / (K_2*rb)
            a_val = (c1 + 4*K_2*c1**2 - rb - 3*K_2*rb**2) / (K_2 * rb)
        else:
            # solve 2K a^2 + a - 2 c1 = 0  -> quadratic for a
            a_val = np.nan
        a_from_b.append(a_val)
    roots_from_elim = np.column_stack([roots_b, np.array(a_from_b, dtype=np.float64)])
    print("poly coeffs:", coeffs)
    print("roots (b, a):")
    for rb, ra in roots_from_elim:
        print(rb, ra)
else:
    print("Sympy not available; skipping elimination.")


## Scan loop